In [97]:
import json
import requests
import pandas as pd
import numpy as np
import sys
import re
import time

from sqlalchemy import create_engine
import pymysql

url = 'https://api.artsy.net/api/tokens/xapp_token'
myobj = {'client_id': '3369c027684c8cf068f9', 'client_secret': 'f9d37e6312fe53c17535d2624bbbcd50', 'total_count':'1'}

resp = requests.post(url, data = myobj)
data = resp.json() # Check the JSON Response Content documentation below
token = data["token"]

1033


In [99]:
i = 0
cursor = ""
nextUrl = ""

while i < total_count:

    if (nextUrl == ""):
        response_artist = requests.get(url = "https://api.artsy.net/api/genes", headers={'X-XAPP-Token': token}, params={"size": genesPerRequest})
    else:
        response_artist = requests.get(url =nextUrl, headers={'X-XAPP-Token': token})
    
    
    data = response_artist.json()
    
    try:
        
        nextUrl = data["_links"]["next"]["href"]
        print(nextUrl)
        
        arrayOfGenes = data["_embedded"]["genes"]
        print("loaded "+str(artistPerRequest)+" / "+ str(len(arrayOfGenes)) +" artist with offset " + str(i)+ " from "+ str(total_count))    
        genesList = pd.DataFrame(columns=['id', 'name', 'display_name', 'description', 'image', 'artworks', 'artists'])

        for gene in arrayOfGenes:
            
            geneData = {"id": gene["id"], 
                                   "name": gene["name"],
                                   "display_name": gene["display_name"],
                                   "description": gene["description"],
                                   "image": gene["_links"]["image"]["href"].replace('{image_version}', "square500"),
                                   "artworks": gene["_links"]["artworks"]["href"],
                                   "artists": gene["_links"]["artists"]["href"]}
            
            genesList = genesList.append(geneData, ignore_index=True)
    
        genesList = genesList.replace(r'^\s*$', np.nan, regex=True)
        genesList = genesList.replace('"', np.nan, regex=True)
        genesList = genesList.replace('nan', np.nan)
        genesList.to_sql('genes', con=sqlEngine, if_exists= "append")
        
        i = i + len(arrayOfGenes)
        
    except KeyError as e: 
        print(data)
        
        time.sleep(1)

https://api.artsy.net/api/genes?cursor=Contemporary+Grotesque%3A4dda6a48fa614f0001002943&size=250
loaded 250 / 250 artist with offset 0 from 1033
https://api.artsy.net/api/genes?cursor=Improbable+Physics%3A53763281cd530e163300008f&size=250
loaded 250 / 250 artist with offset 250 from 1033
https://api.artsy.net/api/genes?cursor=Post-Impressionism%3A4d90d192dcdd5f44a5000065&size=250
loaded 250 / 250 artist with offset 500 from 1033
https://api.artsy.net/api/genes?cursor=Viscous+Forms%3A500808453c585a0002000ae4&size=250
loaded 250 / 250 artist with offset 750 from 1033
https://api.artsy.net/api/genes?cursor=Zoomorphism%3A5523030c7261692d5f093800&size=250
loaded 250 / 33 artist with offset 1000 from 1033
